### this notebook preprocess 240 full traces of VSM-ACTR trails from 6 probelm sets into a format that can be feedinto sentences transformer

In [ ]:
pip install pandas openpyxl

In [ ]:
import sys
if 'google.colab' in sys.modules:  # If in Google Colab environmen

    # Mount google drive to enable access to data files
    from google.colab import drive
    drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C
Mounted at /content/drive


In [ ]:
#Create a base folder to put all trails from 6 problem sets into one folder
import pandas as pd
import os

def convert_text_to_excel(base_folder):
    # Iterate over specified range of folders
    for i in range(6):  # iterate throgh 6 folders
        text_folder = f'{base_folder}/model-output{i}'
        output_folder = f'{base_folder}/excel-output{i}'
        os.makedirs(output_folder, exist_ok=True)
        #change from VSM-ACTR trace format txt into excel
        for filename in os.listdir(text_folder):
            if filename.endswith(".txt"):
                file_path = os.path.join(text_folder, filename)
                df = pd.read_csv(file_path, sep='\t', header=None)
                excel_path = os.path.join(output_folder, filename.replace('.txt', '.xlsx'))
                df.to_excel(excel_path, index=False)

base_folder = '/content/drive/My Drive/model-output'
convert_text_to_excel(base_folder)

In [ ]:
import os
import shutil

In [ ]:
#put all files in to one folder
def organize_excel_files(base_folder, target_folder):

    os.makedirs(target_folder, exist_ok=True)

    # Sequentially move Excel files from each source folder to the target folder
    for i in range(6):
        source_folder = f'{base_folder}/excel-output{i}'
        excel_files = sorted(os.listdir(source_folder))  # Sort to maintain consistent order

        for filename in excel_files:
            if filename.endswith('.xlsx'):
                source_file_path = os.path.join(source_folder, filename)
                target_file_path = os.path.join(target_folder, f'folder{i}_{filename}')

                shutil.move(source_file_path, target_file_path)


# Define the base and target folders
base_folder = '/content/drive/My Drive/model-output'
target_folder = '/content/drive/My Drive/model-output3'
organize_excel_files(base_folder, target_folder)

In [ ]:
import os

folder_path = '/content/drive/My Drive/model_output3'  # Adjust the path if your folder is nested in other folders
excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]


In [ ]:
#data preprocessing
import os
import pandas as pd
from google.colab import drive

def clean_and_save_excel_files(source_folder, target_folder):
    # Ensure the target directory exists
    os.makedirs(target_folder, exist_ok=True)


    excel_files = [f for f in os.listdir(source_folder) if f.endswith('.xlsx')]

    # Iterate over each Excel file
    for excel_file in excel_files:
        file_path = os.path.join(source_folder, excel_file)
        df = pd.read_excel(file_path)

        # Drop columns 'A' and 'B'
        df.drop(columns=['A', 'B'], errors='ignore', inplace=True)

        # Remove rows containing "U(n-1)" or "U(n)"
        df = df[~df.apply(lambda row: row.astype(str).str.contains('U\(n-1\)').any(), axis=1)]
        df = df[~df.apply(lambda row: row.astype(str).str.contains('U\(n\)').any(), axis=1)]

        # Remove rows that contain only numbers
        df = df[~df.apply(lambda row: row.apply(lambda x: pd.to_numeric(x, errors='coerce')).notna().all(), axis=1)]

        # Remove rows that contain only a single word (no spaces)
        df = df[df.apply(lambda row: row.astype(str).apply(lambda x: len(x.split()) > 1).any(), axis=1)]

        df.dropna(inplace=True)  # Remove rows with missing values
        df.drop_duplicates(inplace=True)  # Remove duplicate rows

        # Save the processed DataFrame to the target folder
        processed_filename = os.path.join(target_folder, f"processed_{excel_file}")
        df.to_excel(processed_filename, index=False)

    print("All files have been processed and saved.")

# Define the base and target folders
source_folder = '/content/drive/My Drive/model-output3'
target_folder = '/content/drive/My Drive/model-output4'
clean_and_save_excel_files(source_folder, target_folder)




All files have been processed and saved.


In [ ]:

folder_path = '/content/drive/My Drive/model-output4'

In [ ]:
#change into the format can be feed into sentence transformer
import os
import pandas as pd

# Output sentences list
sentences = []
file_count = 1  # Initialize file count

# Read and format each file
for file_name in sorted(os.listdir(folder_path)):
    if file_name.endswith('.xlsx'):
        full_path = os.path.join(folder_path, file_name)  # Ensure the full path is correctly formed
        df = pd.read_excel(full_path)

        # Generate a single string for each row, joined by commas
        file_sentences = ['"{}"'.format('", "'.join(row.astype(str))) for index, row in df.iterrows()]
        formatted_sentence = ', '.join(file_sentences)
        formatted_sentence_with_label = f"{file_count}: {formatted_sentence}"
        sentences.append(formatted_sentence_with_label)

        # Increment file count for the next file
        file_count += 1

# Join all formatted sentences with a comma separator to form the final output
formatted_sentences = ', '.join(sentences)

# Print or save the output
print(formatted_sentences)




1: "     0.000   GOAL                   SET-BUFFER-CHUNK GOAL GOER NIL", "     0.050   PROCEDURAL             PRODUCTION-FIRED CHOOSE-STRATEGY", "     0.100   PROCEDURAL             PRODUCTION-FIRED DECIDE-BRUTE", "     0.150   PROCEDURAL             PRODUCTION-FIRED BRUTE-DECISION", "assembly is always a good place to reduce time!", "     0.200   PROCEDURAL             PRODUCTION-FIRED REHEADCOUNT", "     0.250   PROCEDURAL             PRODUCTION-FIRED STOP", "this is the end of one decision making", " Utility updates with Reward = -2.0   alpha = 0.2", "  Updating utility of production CHOOSE-STRATEGY", "  Updating utility of production DECIDE-BRUTE", "  Updating utility of production BRUTE-DECISION", "  Updating utility of production REHEADCOUNT", "  Updating utility of production STOP", "     0.300   PROCEDURAL             PRODUCTION-FIRED CHOOSE-STRATEGY", "     0.350   PROCEDURAL             PRODUCTION-FIRED DECIDE-BRUTE", "     0.400   PROCEDURAL             PRODUCTION-FIRED BRUT

In [ ]:
# Save to a text file on Google Drive
output_path = '/content/drive/My Drive/processed_sentences.txt'
with open(output_path, 'w') as f:
    f.write(formatted_sentences)

In [ ]:
#make sure the full traces for each trail being labels by sentencesi (i= 1 to 240 in correspondence to the index of the trails)
import re

structured_data = {}
current_key = None
current_sentences = []

section_pattern = re.compile(r'(\d+): "')

# Read the file
with open('/content/drive/My Drive/processed_sentences.txt', 'r') as file:
    content = file.read()

# Split the content into sections
sections = section_pattern.split(content)[1:]  # Split and remove the first split before the first key

# Process each section
for i in range(0, len(sections), 2):
    key = sections[i].strip()
    sentences_block = sections[i + 1].rstrip('", ')
    sentences = [sentence.strip().strip('"') for sentence in sentences_block.split('", "')]
    if int(key) >= 241 and int(key) <= 440:
        continue

    structured_data[f'sentences{key}'] = sentences

print("Keys parsed and retained in structured_data:")
print(structured_data.keys())


Keys parsed and retained in structured_data:
dict_keys(['sentences1', 'sentences2', 'sentences3', 'sentences4', 'sentences5', 'sentences6', 'sentences7', 'sentences8', 'sentences9', 'sentences10', 'sentences11', 'sentences12', 'sentences13', 'sentences14', 'sentences15', 'sentences16', 'sentences17', 'sentences18', 'sentences19', 'sentences20', 'sentences21', 'sentences22', 'sentences23', 'sentences24', 'sentences25', 'sentences26', 'sentences27', 'sentences28', 'sentences29', 'sentences30', 'sentences31', 'sentences32', 'sentences33', 'sentences34', 'sentences35', 'sentences36', 'sentences37', 'sentences38', 'sentences39', 'sentences40', 'sentences41', 'sentences42', 'sentences43', 'sentences44', 'sentences45', 'sentences46', 'sentences47', 'sentences48', 'sentences49', 'sentences50', 'sentences51', 'sentences52', 'sentences53', 'sentences54', 'sentences55', 'sentences56', 'sentences57', 'sentences58', 'sentences59', 'sentences60', 'sentences61', 'sentences62', 'sentences63', 'sentenc

In [ ]:
# Write the structured data to a file with the specified format
output_file_path = '/content/drive/My Drive/processed_sentences2.txt'
with open(output_file_path, 'w') as file:
    for key, sentences in structured_data.items():
        sentences_str = '", "'.join(sentences)
        file.write(f'{key} = ["{sentences_str}"]\n')

print(f"Structured data has been written to {output_file_path}")

# the processed_sentences2.txt is the data ready to be feed into sentence transformer

Structured data has been written to /content/drive/My Drive/processed_sentences2.txt
